This notebook checks that the reference results for the `ipDataToPointCloud` tool are correct.

In [1]:
import pyvista as pv
import numpy as np

In [2]:
def check_ip_data(mesh, point_cloud, field_name):
    # kept name "sigma_ip", although it might also be "epsilon_ip" or something completely different
    sigma_ip_field = mesh.field_data[field_name]
    sigma_ip_point = point_cloud.point_data[field_name]

    num_ips = sigma_ip_field.shape[0]

    # check that each IP corresponds to a point in the point cloud
    assert num_ips == point_cloud.n_points

    # check that values of sigma_ip are equal
    assert np.allclose(sigma_ip_field, sigma_ip_point, atol=1e-15, rtol=0) 

def check_all_points_enclosed_by_mesh_surface_3D(mesh, point_cloud):
    surface = mesh.extract_surface()
    points_in_mesh = point_cloud.select_enclosed_points(surface)

    # check that each point in the point cloud is contained in the "original" mesh
    assert points_in_mesh.n_points == point_cloud.n_points
    
def check_all_points_inside_bbox_2D(mesh, point_cloud):
    xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds

    assert zmin == 0.0 and zmax == 0.0  # make sure we are in 2D
    
    assert np.all(point_cloud.points[:,0] > xmin)
    assert np.all(point_cloud.points[:,0] < xmax)
    assert np.all(point_cloud.points[:,1] > ymin)
    assert np.all(point_cloud.points[:,1] < ymax)

# Mixed element test

In [3]:
mesh = pv.read("mesh_with_3D_different_elements_sigma_ip.vtu")
point_cloud = pv.read("mesh_with_3D_different_elements_sigma_ip_point_cloud.vtu")

check_ip_data(mesh, point_cloud, "sigma_ip")
check_all_points_enclosed_by_mesh_surface_3D(mesh, point_cloud)

# Tri and quad mesh test

In [4]:
mesh = pv.read("TriQuad/quad_tri_THM_t_864000_000000.vtu")
point_cloud = pv.read("TriQuad/quad_tri_THM_t_864000_000000_point_cloud.vtu")

check_ip_data(mesh, point_cloud, "sigma_ip")
check_ip_data(mesh, point_cloud, "epsilon_ip")

check_all_points_inside_bbox_2D(mesh, point_cloud)